In [1]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

Using TensorFlow backend.
/Users/akansal/anaconda3/envs/tensorflow/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)


In [22]:
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

img = load_img('data/train/cats/cat.10.jpg')
x = img_to_array(img)
x = x.reshape((1, ) + x.shape)

In [23]:
x.shape

(1, 499, 489, 3)

In [18]:
i = 0
for batch in datagen.flow(x, batch_size=1,
                         save_to_dir='preview',
                         save_prefix='cat',
                         save_format='jpeg'):
    i += 1
    if i > 20:
        break

In [2]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(150, 150, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Conv2D(32, (3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
activation_1 (Activation)    (None, 148, 148, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 72, 72, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 72, 72, 32)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 34, 34, 64)        18496     
__________

In [3]:
model.compile(loss='binary_crossentropy',
             optimizer='rmsprop',
             metrics=['accuracy'])

In [4]:
batch_size=16

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    'data/train',
    target_size=(150, 150),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    'data/validation',
    target_size=(150, 150),
    batch_size=batch_size,
    class_mode='binary')

Found 2000 images belonging to 2 classes.
Found 800 images belonging to 2 classes.


In [46]:

from keras.callbacks import ModelCheckpoint
import os

save_checkpoint = ModelCheckpoint('weights-best.hdf5', monitor='val_acc', verbose=1, save_best_only=True, mode='max')

try:
    model.load_weights('weights-best.hdf5')
except OSError:
    pass

hist = model.fit_generator(
    train_generator,
    steps_per_epoch= 2000 // batch_size,
    epochs = 50,
    validation_data=validation_generator,
    validation_steps=800 // batch_size,
    callbacks=[save_checkpoint])


Epoch 1/50
125/125 [==============================] - 73s 582ms/step - loss: 0.5632 - acc: 0.7210 - val_loss: 0.5333 - val_acc: 0.7225

Epoch 00001: val_acc improved from -inf to 0.72250, saving model to weights-best.hdf5
Epoch 2/50
125/125 [==============================] - 84s 676ms/step - loss: 0.5442 - acc: 0.7375 - val_loss: 0.5395 - val_acc: 0.7175

Epoch 00002: val_acc did not improve from 0.72250
Epoch 3/50
125/125 [==============================] - 74s 593ms/step - loss: 0.5497 - acc: 0.7450 - val_loss: 0.5336 - val_acc: 0.7250

Epoch 00003: val_acc improved from 0.72250 to 0.72500, saving model to weights-best.hdf5
Epoch 4/50
125/125 [==============================] - 74s 588ms/step - loss: 0.5300 - acc: 0.7495 - val_loss: 0.5489 - val_acc: 0.7100

Epoch 00004: val_acc did not improve from 0.72500
Epoch 5/50
125/125 [==============================] - 72s 574ms/step - loss: 0.5379 - acc: 0.7420 - val_loss: 0.5417 - val_acc: 0.7350

Epoch 00005: val_acc improved from 0.72500 to

KeyboardInterrupt: 

In [57]:
from keras import applications
datagen = ImageDataGenerator(rescale=1./255)

model = applications.VGG16(include_top=False, weights='imagenet')

generator = datagen.flow_from_directory(
    'data/train',
    target_size=(150, 150),
    batch_size=1,
    class_mode=None,
    shuffle=None)

bottleneck_features_train = model.predict_generator(
    generator, 2000, verbose=1)

np.save(open('bottleneck_train_features.npy', 'w'), bottleneck_features_train)

generator = datagen.flow_from_directory(
    'data/validation',
    target_size=(150, 150),
    batch_size=1,
    class_mode=None,
    shuffle=False)
bottleneck_features_validation = model.predict_generator(
    generator, 800, verbose=1)

np.save(open('bottleneck_validation_features.npy', 'w'), bottleneck_features_validation)


Found 2000 images belonging to 2 classes.
2000/2000 [==============================] - 361s 180ms/step
Found 800 images belonging to 2 classes.
800/800 [==============================] - 149s 186ms/step


In [64]:
from keras.callbacks import ModelCheckpoint
train_data = np.load(open('bottleneck_train_features.npy', 'rb'))
train_labels = np.array([0] * 1000 + [1] * 1000)

validation_data = np.load(open('bottleneck_validation_features.npy', 'rb'))
validation_labels = np.array([0] * 400 + [1] * 400)

model = Sequential()
model.add(Flatten(input_shape=train_data.shape[1:]))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])


save_checkpoint = ModelCheckpoint('weights-best-vgg.hdf5', monitor='val_acc', verbose=1, save_best_only=True, mode='max')
model.fit(train_data, train_labels,
         epochs=50,
         batch_size=16,
         validation_data=(validation_data, validation_labels),
         callbacks=[save_checkpoint])



Train on 2000 samples, validate on 800 samples
Epoch 1/50
2000/2000 [==============================] - 4s 2ms/step - loss: 0.7234 - acc: 0.7620 - val_loss: 0.2822 - val_acc: 0.8812

Epoch 00001: val_acc improved from -inf to 0.88125, saving model to weights-best-vgg.hdf5
Epoch 2/50
2000/2000 [==============================] - 4s 2ms/step - loss: 0.3384 - acc: 0.8495 - val_loss: 0.3945 - val_acc: 0.8375

Epoch 00002: val_acc did not improve from 0.88125
Epoch 3/50
2000/2000 [==============================] - 4s 2ms/step - loss: 0.3040 - acc: 0.8675 - val_loss: 0.2610 - val_acc: 0.8888

Epoch 00003: val_acc improved from 0.88125 to 0.88875, saving model to weights-best-vgg.hdf5
Epoch 4/50
2000/2000 [==============================] - 4s 2ms/step - loss: 0.2316 - acc: 0.9020 - val_loss: 0.2898 - val_acc: 0.8900

Epoch 00004: val_acc improved from 0.88875 to 0.89000, saving model to weights-best-vgg.hdf5
Epoch 5/50
2000/2000 [==============================] - 4s 2ms/step - loss: 0.2181 - ac

2000/2000 [==============================] - 3s 2ms/step - loss: 0.0175 - acc: 0.9960 - val_loss: 0.8159 - val_acc: 0.8950

Epoch 00043: val_acc did not improve from 0.90750
Epoch 44/50
2000/2000 [==============================] - 3s 2ms/step - loss: 0.0249 - acc: 0.9920 - val_loss: 0.8421 - val_acc: 0.8888

Epoch 00044: val_acc did not improve from 0.90750
Epoch 45/50
2000/2000 [==============================] - 3s 2ms/step - loss: 0.0152 - acc: 0.9965 - val_loss: 0.8698 - val_acc: 0.8888

Epoch 00045: val_acc did not improve from 0.90750
Epoch 46/50
2000/2000 [==============================] - 3s 2ms/step - loss: 0.0171 - acc: 0.9940 - val_loss: 0.8248 - val_acc: 0.8975

Epoch 00046: val_acc did not improve from 0.90750
Epoch 47/50
2000/2000 [==============================] - 3s 2ms/step - loss: 0.0102 - acc: 0.9965 - val_loss: 0.8805 - val_acc: 0.8925

Epoch 00047: val_acc did not improve from 0.90750
Epoch 48/50
2000/2000 [==============================] - 3s 2ms/step - loss: 0.023

In [65]:
model.load_weights('weights-best-vgg.hdf5')
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory('data/test_single', target_size=(150, 150),
    batch_size=1,
    class_mode=None, 
    shuffle=False)

predict = model.predict_generator(test_generator, steps=len(test_generator.filenames))

Found 5 images belonging to 1 classes.


ValueError: Error when checking input: expected flatten_3_input to have shape (4, 4, 512) but got array with shape (150, 150, 3)